In [3]:
import requests
from bs4 import BeautifulSoup

In [4]:
base_url = 'https://247sports.com/'
url = base_url + "Season/2021-Football/CollegeTeamTalentComposite/?Conference=";
conferences = ['ACC','Big-12','Big-Ten','Pac-12','SEC','AAC']

In [62]:
def parse_team_info(soup):
    a = soup.find('a', {'class':'rankings-page__name-link'})
    teamName = a.string.strip()
    team_breakdown_url = a['href'] if a.has_attr('href') else None
    logo_container = soup.find('img')
    logo_url = logo_container['data-src'] if logo_container.has_attr('data-src') else None
    if logo_url is not None:
        logo_url = logo_url.split('?')[0]
    
    print(teamName)
    print(team_breakdown_url)
    print(logo_url)
    print("--------")

In [63]:
for conference in conferences:
    print("#####    " + conference + "  ######")
    conference_url = url + conference
    html = requests.get(conference_url, headers={"User-Agent": "Requests"}).content
    soup = BeautifulSoup(html, 'html.parser')
    team_info_containers = soup.find_all('li', {'class' : 'rankings-page__list-item'})
    for team_container in team_info_containers:
        parse_team_info(team_container)
    

#####    ACC  ######
Clemson
/Team/Clemson-3/Roster
https://s3media.247sports.com/Uploads/Assets/565/682/4682565.png
--------
Miami
/Team/Miami-13/Roster
https://s3media.247sports.com/Uploads/Assets/201/648/4648201.png
--------
North Carolina
/Team/North-Carolina-15/Roster
https://s3media.247sports.com/Uploads/Assets/602/120/8120602.png
--------
Florida State
/Team/Florida-State-7/Roster
https://s3media.247sports.com/Uploads/Assets/414/682/4682414.png
--------
Georgia Tech
/Team/Georgia-Tech-9/Roster
https://s3media.247sports.com/Uploads/Assets/197/648/4648197.png
--------
NC State
/Team/NC-State-17/Roster
https://s3media.247sports.com/Uploads/Assets/728/975/7975728.png
--------
Pittsburgh
/Team/Pittsburgh-460/Roster
https://s3media.247sports.com/Uploads/Assets/109/167/9167109.png
--------
Virginia Tech
/Team/Virginia-Tech-21/Roster
https://s3media.247sports.com/Uploads/Assets/212/648/4648212.png
--------
Boston College
/Team/Boston-College-1/Roster
https://s3media.247sports.com/Upload